In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import datetime
from numpy import hstack

In [3]:
import tensorflow

In [108]:
#obtain list of dates
directory = "Golden State Warriors"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [109]:
# create dictionary 
final_data = {}

In [110]:
opponents = {}
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"{directory}/{i}", index_col=0)
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]
    
    # (error for now) opponents[date] = data["Opponent"][0]
    
    data[date] = data["Score"]
    final_data[date] = data[date]
    final_data[date].append

In [111]:
# convert dict to dataframe
data_df = pd.DataFrame(final_data)
data_df

,10/18/2022,10/25/2022,10/23/2022,10/21/2022
Andrew Wiggins,39.00,33.00,42.75,46.50
Donte DiVincenzo,14.00,NaN,4.50,3.50
Draymond Green,17.25,35.00,17.00,38.25
JaMychal Green,19.75,4.75,14.00,15.75
James Wiseman,18.75,19.75,25.00,18.25
Jonathan Kuminga,4.75,7.25,3.50,5.50
Jordan Poole,28.00,27.50,33.00,22.75
Kevon Looney,19.50,15.00,27.00,11.25
Klay Thompson,27.00,7.00,18.00,25.50
Moses Moody,6.75,24.75,5.25,7.75


In [45]:
tester.to_csv("Final_Score/Tables/Boston_Celtics.csv")

In [112]:
opp = pd.DataFrame.transpose(pd.DataFrame.from_dict(opponents, orient="index"))
data_df = data_df.append(opp)

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_40924/1654206437.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(opp)


In [113]:

data_df

,10/18/2022,10/25/2022,10/23/2022,10/21/2022
Andrew Wiggins,39.00,33.00,42.75,46.50
Donte DiVincenzo,14.00,NaN,4.50,3.50
Draymond Green,17.25,35.00,17.00,38.25
JaMychal Green,19.75,4.75,14.00,15.75
James Wiseman,18.75,19.75,25.00,18.25
Jonathan Kuminga,4.75,7.25,3.50,5.50
Jordan Poole,28.00,27.50,33.00,22.75
Kevon Looney,19.50,15.00,27.00,11.25
Klay Thompson,27.00,7.00,18.00,25.50
Moses Moody,6.75,24.75,5.25,7.75


In [115]:
tester["Datetime"] = pd.to_datetime(data_df.columns.tolist()).tolist()

In [114]:
tester = pd.DataFrame.transpose(data_df)

In [116]:
#create dict
final_result = {}

In [117]:
tester

,Andrew Wiggins,Donte DiVincenzo,Draymond Green,JaMychal Green,James Wiseman,Jonathan Kuminga,Jordan Poole,Kevon Looney,Klay Thompson,Moses Moody,Ryan Rollins,Stephen Curry,Ty Jerome,Datetime
10/18/2022,39.00,14.0,17.25,19.75,18.75,4.75,28.00,19.50,27.0,6.75,2.75,61.00,NaN,2022-10-18
10/25/2022,33.00,NaN,35.00,4.75,19.75,7.25,27.50,15.00,7.0,24.75,NaN,45.25,4.25,2022-10-25
10/23/2022,42.75,4.5,17.00,14.00,25.00,3.50,33.00,27.00,18.0,5.25,0.00,46.75,NaN,2022-10-23
10/21/2022,46.50,3.5,38.25,15.75,18.25,5.50,22.75,11.25,25.5,7.75,NaN,47.25,NaN,2022-10-21


In [118]:
tester = tester.sort_values(by="Datetime")

In [119]:
tester.columns

Index(['Andrew Wiggins', 'Donte DiVincenzo', 'Draymond Green',
       'JaMychal Green', 'James Wiseman', 'Jonathan Kuminga', 'Jordan Poole',
       'Kevon Looney', 'Klay Thompson', 'Moses Moody', 'Ryan Rollins',
       'Stephen Curry', 'Ty Jerome', 'Datetime'],
      dtype='object')

In [120]:
columns = tester.columns.tolist()[:-1]

In [121]:
columns

['Andrew Wiggins',
 'Donte DiVincenzo',
 'Draymond Green',
 'JaMychal Green',
 'James Wiseman',
 'Jonathan Kuminga',
 'Jordan Poole',
 'Kevon Looney',
 'Klay Thompson',
 'Moses Moody',
 'Ryan Rollins',
 'Stephen Curry',
 'Ty Jerome']

In [122]:
add = {}

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 3
    if len(rent_rate) < 4:
        continue
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(rent_rate[-3:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Andrew Wiggins
[39.   46.5  42.75] 33.0


2022-10-27 12:57:19.209577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Donte DiVincenzo
Draymond Green
[17.25 38.25 17.  ] 35.0


2022-10-27 12:57:27.617101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-27 12:57:29.118926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


JaMychal Green
[19.75 15.75 14.  ] 4.75


...

2022-10-27 12:58:30.070420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-27 12:58:30.826189: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Ryan Rollins
Stephen Curry
[61.   47.25 46.75] 45.25


2022-10-27 12:58:38.095625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-27 12:58:38.775415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Ty Jerome


2022-10-27 12:58:44.947160: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [123]:
add

{'Andrew Wiggins': 34.251426696777344,
 'Draymond Green': 45.56275939941406,
 'JaMychal Green': 3.0609183311462402,
 'James Wiseman': 21.661008834838867,
 'Jonathan Kuminga': 8.004239082336426,
 'Jordan Poole': 27.125572204589844,
 'Kevon Looney': 14.756599426269531,
 'Klay Thompson': 4.8371076583862305,
 'Moses Moody': 33.25953674316406,
 'Stephen Curry': 38.6179084777832}

In [124]:
tester

,Andrew Wiggins,Donte DiVincenzo,Draymond Green,JaMychal Green,James Wiseman,Jonathan Kuminga,Jordan Poole,Kevon Looney,Klay Thompson,Moses Moody,Ryan Rollins,Stephen Curry,Ty Jerome,Datetime
10/18/2022,39.00,14.0,17.25,19.75,18.75,4.75,28.00,19.50,27.0,6.75,2.75,61.00,NaN,2022-10-18
10/21/2022,46.50,3.5,38.25,15.75,18.25,5.50,22.75,11.25,25.5,7.75,NaN,47.25,NaN,2022-10-21
10/23/2022,42.75,4.5,17.00,14.00,25.00,3.50,33.00,27.00,18.0,5.25,0.00,46.75,NaN,2022-10-23
10/25/2022,33.00,NaN,35.00,4.75,19.75,7.25,27.50,15.00,7.0,24.75,NaN,45.25,4.25,2022-10-25


In [125]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')

In [126]:
Data

,0
Andrew Wiggins,34.251427
Draymond Green,45.562759
JaMychal Green,3.060918
James Wiseman,21.661009
Jonathan Kuminga,8.004239
Jordan Poole,27.125572
Kevon Looney,14.756599
Klay Thompson,4.837108
Moses Moody,33.259537
Stephen Curry,38.617908


In [127]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/{directory}.csv", index=True)

Function below used for multiple teams

In [37]:
#obtain list of dates
directorys = ["Los Angeles Lakers"]


for directory in directorys:
    list_dates = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            list_dates.append(filename)
    # create dictionary 
    final_data = {}
    
    # reads in each csv
    for i in list_dates:
        data = pd.read_csv(f"{directory}/{i}", index_col=0)

        #gathers data and adds to dict
        date = data["Date"].tolist()[0]
        data[date] = data["Score"]
        final_data[date] = data[date]
    # convert dict to dataframe
    data_df = pd.DataFrame(final_data)
    
    
    tester = pd.DataFrame.transpose(data_df)
    columns = tester.columns.tolist()
    
    #create dict
    final_result = {}
    
    add = {}

    #iterate through index
    for column in columns:
        print(column)

    # univariate data prep LSTM
        from numpy import array

        # split sequence
        def split_sequence(sequence, n_steps):
            X, y = list(), list()
            for i in range(len(sequence)):
                # find end of pattern 
                end_ix = i + n_steps
                #check if beyond sequence to break
                if end_ix > len(sequence)-1:
                    break
                #gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return array(X), array(y)
        # sequence 
        rent_rate = list(tester[column].dropna())
        # number of steps
        n_steps = 3
        if len(rent_rate) < 4:
            continue
        X, y = split_sequence(rent_rate, n_steps)
        # summarize the data
        for i in range(len(X)):
            print(X[i], y[i])

        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # fit model
        model.fit(X, y, epochs=200, verbose=0)

        # demonstrate prediction
        x_input = array(rent_rate[-3:])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        add[column] = yhat.tolist()[0][0]
    # convert dict to dataframe
    Data = pd.DataFrame.from_dict(add, orient='index')

    
    # convert dataframe to csv
    mypath = f"Final_Score/LSTM/{directory}"
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    Data.to_csv(f"{mypath}/{directory}.csv", index=True)

Anthony Davis
Austin Reaves
[72.   36.75 17.75] 20.5
[36.75 17.75 20.5 ] 7.5
[17.75 20.5   7.5 ] 21.25
[20.5   7.5  21.25] 20.5


2022-10-15 16:23:25.183502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Cole Swider
[15.75 16.   30.75] 11.0
[16.   30.75 11.  ] 14.5
[30.75 11.   14.5 ] 12.5


2022-10-15 16:23:28.359052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:23:28.826176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


D.J. Augustin
Damian Jones
[9.   8.   1.25] 11.0
[ 8.    1.25 11.  ] 11.5
[ 1.25 11.   11.5 ] 5.25


...

Nate Pierre-Louis
Patrick Beverley
[19.   27.   13.25] 7.25


2022-10-15 16:23:55.531063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:23:56.448071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Russell Westbrook
[23.   15.25 11.5 ] 3.25


2022-10-15 16:23:59.827308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:24:00.300948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

Backtesting

In [12]:
list_dates

['06-16.csv',
 '06-02.csv',
 '06-10.csv',
 '06-05.csv',
 '06-13.csv',
 '04-17.csv',
 '05-25.csv',
 '05-19.csv',
 '05-27.csv',
 '05-23.csv',
 '05-21.csv',
 '05-09.csv',
 '05-11.csv',
 '05-07.csv',
 '05-13.csv',
 '05-17.csv',
 '05-03.csv',
 '05-29.csv',
 '05-15.csv',
 '05-01.csv',
 '06-08.csv',
 '04-25.csv',
 '04-20.csv',
 '04-23.csv']

In [13]:
collect = []
for name in opponents:
    if opponents[name] not in collect:
        collect.append(opponents[name])

In [14]:
collect

['Golden State Warriors', 'Brooklyn Nets', 'Miami Heat', 'Milwaukee Bucks']

In [15]:
data_df

,06/16/2022,06/02/2022,06/10/2022,06/05/2022,06/13/2022,04/17/2022,05/25/2022,05/19/2022,05/27/2022,05/23/2022,...,05/13/2022,05/17/2022,05/03/2022,05/29/2022,05/15/2022,05/01/2022,06/08/2022,04/25/2022,04/20/2022,04/23/2022
Aaron Nesmith,0.00,0.00,NaN,11.50,4.25,NaN,0.00,1.25,0.00,10.25,...,1.25,8.75,1.25,NaN,4.75,2.75,1.25,NaN,NaN,NaN
Al Horford,46.00,42.00,24.00,14.50,21.75,44.25,38.25,21.75,20.75,34.25,...,28.50,NaN,39.25,33.50,29.50,36.00,34.50,23.00,34.00,15.25
Daniel Theis,NaN,5.50,NaN,9.25,NaN,13.00,NaN,4.75,NaN,7.75,...,6.50,18.00,NaN,NaN,9.75,6.00,NaN,20.50,26.50,20.00
Derrick White,5.75,28.25,20.75,25.50,8.75,22.25,27.25,NaN,43.25,39.00,...,22.75,10.00,13.50,14.75,9.50,16.50,14.75,19.50,8.25,12.50
Grant Williams,7.25,8.25,4.75,11.25,11.25,13.00,12.50,27.00,6.50,16.25,...,13.25,12.75,33.25,19.50,41.50,16.00,18.25,26.75,30.00,16.75
Jaylen Brown,49.25,45.25,33.50,29.50,34.75,45.25,36.00,40.00,37.50,23.75,...,37.25,44.00,54.50,39.00,33.00,31.25,48.75,38.00,41.00,38.50
Jayson Tatum,33.25,39.25,52.75,43.00,46.00,53.00,49.00,43.25,51.75,51.50,...,66.75,55.50,53.25,54.50,43.50,42.00,49.50,39.25,43.00,65.25
Juwan Morgan,0.00,0.00,NaN,NaN,1.25,NaN,0.00,2.25,NaN,0.00,...,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN
Luke Kornet,0.75,NaN,NaN,3.50,4.75,NaN,0.00,4.50,NaN,NaN,...,NaN,NaN,1.25,NaN,0.00,0.00,NaN,NaN,NaN,NaN
Malik Fitts,NaN,0.00,NaN,3.50,NaN,NaN,0.00,NaN,NaN,0.00,...,NaN,NaN,1.25,NaN,3.50,3.25,0.00,NaN,NaN,NaN


In [16]:
directorys

NameError: name 'directorys' is not defined

In [28]:
#obtain list of dates
directorys = collect


for directory in directorys:
    print(directory)
    list_dates = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            list_dates.append(filename)
    # create dictionary 
    final_data = {}
    
    # reads in each csv
    for i in list_dates:
        data = pd.read_csv(f"{directory}/{i}", index_col=0)

        #gathers data and adds to dict
        date = data["Date"].tolist()[0]
        data[date] = data["Score"]
        final_data[date] = data[date]
    # convert dict to dataframe
    opp = pd.DataFrame(final_data)
    
    
    opp_df = pd.DataFrame.transpose(opp)
    
    opp_df["Datetime"] = pd.to_datetime(opp.columns.tolist()).tolist()
    
    columns = opp_df.columns.tolist()[:-1]
    
    main_test = tester.columns.tolist()[:-2]
    
    for date in tester.loc[tester[0] == directory]["Datetime"]:
        print(date)
        main = tester.loc[tester["Datetime"] < date]
        results = opp_df.loc[opp_df["Datetime"] < date]
    
    
        #create dict
        final_result = {}

        add = {}

        #iterate through index
        for column in columns:
            print(column)

        # univariate data prep LSTM
            from numpy import array

            # split sequence
            def split_sequence(sequence, n_steps):
                X, y = list(), list()
                for i in range(len(sequence)):
                    # find end of pattern 
                    end_ix = i + n_steps
                    #check if beyond sequence to break
                    if end_ix > len(sequence)-1:
                        break
                    #gather input and output parts of the pattern
                    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                    X.append(seq_x)
                    y.append(seq_y)
                return array(X), array(y)
            # sequence 
            rent_rate = list(results[column].dropna())
            # number of steps
            n_steps = 5
            if len(rent_rate) < 5:
                continue
            X, y = split_sequence(rent_rate, n_steps)
            # summarize the data
            for i in range(len(X)):
                print(X[i], y[i])

            # reshape from [samples, timesteps] into [samples, timesteps, features]
            n_features = 1
            X = X.reshape((X.shape[0], X.shape[1], n_features))

            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            # fit model
            model.fit(X, y, epochs=200, verbose=0)

            # demonstrate prediction
            x_input = array(rent_rate[-5:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            add[column] = yhat.tolist()[0][0]
        # convert dict to dataframe
        Data = pd.DataFrame.from_dict(add, orient='index')


        # convert dataframe to csv
        mypath = f"Final_Score/LSTM/Backtesting/{directory}"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)
        Data.to_csv(f"{mypath}/{directory}_{date}.csv", index=True)
        
        #create dict
        final_result = {}

        add = {}

        
        #iterate through index
        for column in main_test:
            print(column)

        # univariate data prep LSTM
            from numpy import array

            # split sequence
            def split_sequence(sequence, n_steps):
                X, y = list(), list()
                for i in range(len(sequence)):
                    # find end of pattern 
                    end_ix = i + n_steps
                    #check if beyond sequence to break
                    if end_ix > len(sequence)-1:
                        break
                    #gather input and output parts of the pattern
                    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                    X.append(seq_x)
                    y.append(seq_y)
                return array(X), array(y)
            # sequence 
            rent_rate = list(main[column].dropna())
            # number of steps
            n_steps = 5
            if len(rent_rate) < 5:
                continue
            X, y = split_sequence(rent_rate, n_steps)
            # summarize the data
            for i in range(len(X)):
                print(X[i], y[i])

            # reshape from [samples, timesteps] into [samples, timesteps, features]
            n_features = 1
            X = X.reshape((X.shape[0], X.shape[1], n_features))

            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            # fit model
            model.fit(X, y, epochs=200, verbose=0)

            # demonstrate prediction
            x_input = array(rent_rate[-5:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            add[column] = yhat.tolist()[0][0]
        # convert dict to dataframe
        Data = pd.DataFrame.from_dict(add, orient='index')


        # convert dataframe to csv
        mypath = f"Final_Score/LSTM/Backtesting/Boston"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)
        Data.to_csv(f"{mypath}/Boston_{date}.csv", index=True)

NameError: name 'collect' is not defined

In [25]:
X

array([], dtype=float64)

## Backtest individual teams

In [ ]:
#obtain list of dates
directorys = ["Boston Celtics"]


for directory in directorys:
    list_dates = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            list_dates.append(filename)
    # create dictionary 
    final_data = {}
    
    # reads in each csv
    for i in list_dates:
        data = pd.read_csv(f"{directory}/{i}", index_col=0)

        #gathers data and adds to dict
        date = data["Date"].tolist()[0]
        data[date] = data["Score"]
        final_data[date] = data[date]
    # convert dict to dataframe
    data_df = pd.DataFrame(final_data)
    
    
    tester = pd.DataFrame.transpose(data_df)
    columns = tester.columns.tolist()
    
    #create dict
    final_result = {}
    
    add = {}

    #iterate through index
    for column in columns:
        print(column)

    # univariate data prep LSTM
        from numpy import array

        # split sequence
        def split_sequence(sequence, n_steps):
            X, y = list(), list()
            for i in range(len(sequence)):
                # find end of pattern 
                end_ix = i + n_steps
                #check if beyond sequence to break
                if end_ix > len(sequence)-1:
                    break
                #gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return array(X), array(y)
        # sequence 
        rent_rate = list(tester[column].dropna())
        # number of steps
        n_steps = 3
        if len(rent_rate) < 5:
            continue
        X, y = split_sequence(rent_rate, n_steps)
        # summarize the data
        for i in range(len(X)):
            print(X[i], y[i])

        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # fit model
        model.fit(X, y, epochs=200, verbose=0)

        # demonstrate prediction
        x_input = array(rent_rate[-3:])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        add[column] = yhat.tolist()[0][0]
    # convert dict to dataframe
    Data = pd.DataFrame.from_dict(add, orient='index')

    
    # convert dataframe to csv
    mypath = f"Final_Score/LSTM/{directory}"
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    Data.to_csv(f"{mypath}/{directory}.csv", index=True)

# Multiple Input Series

In [61]:
eff = pd.read_csv("Team_defensive_efficency.csv")

efficiency = {}

for name in opponents:
    team = opponents[name]
    team_name = team.split(" ")
    
    team_opp = team_name[0]
    
    if team_name[0] == ("Golden" or "New"):
        team_opp = f"{team_name[0]} {team_name[1]}"
    
    if team_name[0] == "Los":
        team_opp = f"LA {team_name[2]}"

    
    efficiency[name] = eff.loc[eff["Unnamed: 0"] == team_opp]["2021"].values[0]
    
eff_df = pd.DataFrame.transpose(pd.DataFrame.from_dict(efficiency, orient="index"))

eff_df.index = ["Efficiency"]

data_efficiency_df = data_df.append(eff_df)

data_efficiency_df.tail()

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_28858/1235662510.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_efficiency_df = data_df.append(eff_df)


,06/16/2022,06/02/2022,06/10/2022,06/05/2022,06/13/2022,04/17/2022,05/25/2022,05/19/2022,05/27/2022,05/23/2022,...,05/13/2022,05/17/2022,05/03/2022,05/29/2022,05/15/2022,05/01/2022,06/08/2022,04/25/2022,04/20/2022,04/23/2022
Payton Pritchard,2.25,19.5,1.75,13.5,0.75,4.75,1.25,18.5,0.0,26.0,...,3.5,33.75,14.75,0.0,26.75,9.5,4.5,4.25,16.0,14.5
Robert Williams III,30.75,24.5,34.0,10.0,23.0,NaN,26.5,17.5,18.75,27.25,...,NaN,34.25,20.0,6.75,NaN,24.0,34.5,10.75,NaN,8.0
Sam Hauser,0.0,0.0,NaN,5.0,1.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.25,NaN,NaN,0.0,0.0,NaN,NaN,NaN
0,Golden State Warriors,Golden State Warriors,Golden State Warriors,Golden State Warriors,Golden State Warriors,Brooklyn Nets,Miami Heat,Miami Heat,Miami Heat,Miami Heat,...,Milwaukee Bucks,Miami Heat,Milwaukee Bucks,Miami Heat,Milwaukee Bucks,Milwaukee Bucks,Golden State Warriors,Brooklyn Nets,Brooklyn Nets,Brooklyn Nets
Efficiency,1.042,1.042,1.042,1.042,1.042,1.093,1.052,1.052,1.052,1.052,...,1.075,1.052,1.075,1.052,1.075,1.075,1.042,1.093,1.093,1.093


In [65]:
tester_eff = pd.DataFrame.transpose(data_efficiency_df)
tester_eff.drop(columns=[0], inplace=True)
tester_eff.head()

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser,Efficiency
06/16/2022,0.0,46.0,NaN,5.75,7.25,49.25,33.25,0.0,0.75,NaN,33.0,0.0,2.25,30.75,0.0,1.042
06/02/2022,0.0,42.0,5.5,28.25,8.25,45.25,39.25,0.0,NaN,0.0,36.25,0.0,19.5,24.5,0.0,1.042
06/10/2022,NaN,24.0,NaN,20.75,4.75,33.5,52.75,NaN,NaN,NaN,39.0,NaN,1.75,34.0,NaN,1.042
06/05/2022,11.5,14.5,9.25,25.5,11.25,29.5,43.0,NaN,3.5,3.5,11.5,8.75,13.5,10.0,5.0,1.042
06/13/2022,4.25,21.75,NaN,8.75,11.25,34.75,46.0,1.25,4.75,NaN,26.25,0.0,0.75,23.0,1.5,1.042


In [90]:
tester_eff["Aaron Nesmith"].dropna()

06/16/2022      0.0
06/02/2022      0.0
06/05/2022     11.5
06/13/2022     4.25
05/25/2022      0.0
05/19/2022     1.25
05/27/2022      0.0
05/23/2022    10.25
05/21/2022      0.0
05/13/2022     1.25
05/17/2022     8.75
05/03/2022     1.25
05/15/2022     4.75
05/01/2022     2.75
06/08/2022     1.25
Name: Aaron Nesmith, dtype: object

In [97]:
hstack((rent_rate, eff, predict)) 

array([[0.0, 1.042, 0.0],
       [0.0, 1.042, 0.0],
       [11.5, 1.042, 11.5],
       [4.25, 1.042, 4.25],
       [0.0, 1.052, 0.0],
       [1.25, 1.052, 1.25],
       [0.0, 1.052, 0.0],
       [10.25, 1.052, 10.25],
       [0.0, 1.052, 0.0],
       [1.25, 1.075, 1.25],
       [8.75, 1.052, 8.75],
       [1.25, 1.075, 1.25],
       [4.75, 1.075, 4.75],
       [2.75, 1.075, 2.75],
       [1.25, 1.042, 1.25]], dtype=object)

In [93]:
eff

[1.042,
 1.042,
 1.042,
 1.042,
 1.052,
 1.052,
 1.052,
 1.052,
 1.052,
 1.075,
 1.052,
 1.075,
 1.075,
 1.075,
 1.042]

In [179]:
add = {}

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence):
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    df = tester_eff[[column, "Efficiency"]].dropna()
    
    name = df.columns[0]
    
    rent_rate = array(df[name])
    
    eff = array(df["Efficiency"])
    
    predict = rent_rate
    
    
    # convert to [rows, columns] structure
    rent_rate = rent_rate.reshape((len(rent_rate), 1))
    eff = eff.reshape((len(eff), 1))
    predict = predict.reshape((len(predict), 1))
    
    # number of steps
    n_steps = 3
    if len(rent_rate) < 5:
        continue   
        
    # horizontally stack columns
    dataset = hstack((rent_rate, eff, predict))    
        
    X, y = split_sequence(dataset, n_steps)
    
    X = X.astype(float)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = X.shape[2]

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    
    rent = []
    pred = []
    for i in rent_rate[-3:]:
        rent.append(i[0])
        pred.append(i[0])
    eff_pred = []
    for i in eff[-3:]:
        eff_pred.append(i[0])

    # demonstrate prediction
    x_input = array([[rent[0], eff_pred[0]], [rent[1], eff_pred[1]], [rent[2], eff_pred[2]]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Aaron Nesmith
[[ 0.     1.042]
 [ 0.     1.042]
 [11.5    1.042]] 11.5
[[ 0.     1.042]
 [11.5    1.042]
 [ 4.25   1.042]] 4.25
[[11.5    1.042]
 [ 4.25   1.042]
 [ 0.     1.052]] 0.0
[[4.25  1.042]
 [0.    1.052]
 [1.25  1.052]] 1.25
[[0.    1.052]
 [1.25  1.052]
 [0.    1.052]] 0.0
[[ 1.25   1.052]
 [ 0.     1.052]
 [10.25   1.052]] 10.25
[[ 0.     1.052]
 [10.25   1.052]
 [ 0.     1.052]] 0.0
[[10.25   1.052]
 [ 0.     1.052]
 [ 1.25   1.075]] 1.25
[[0.    1.052]
 [1.25  1.075]
 [8.75  1.052]] 8.75
[[1.25  1.075]
 [8.75  1.052]
 [1.25  1.075]] 1.25
[[8.75  1.052]
 [1.25  1.075]
 [4.75  1.075]] 4.75
[[1.25  1.075]
 [4.75  1.075]
 [2.75  1.075]] 2.75
[[4.75  1.075]
 [2.75  1.075]
 [1.25  1.042]] 1.25


2022-09-10 21:42:47.543408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Al Horford
[[46.     1.042]
 [42.     1.042]
 [24.     1.042]] 24.0
[[42.     1.042]
 [24.     1.042]
 [14.5    1.042]] 14.5
[[24.     1.042]
 [14.5    1.042]
 [21.75   1.042]] 21.75
[[14.5    1.042]
 [21.75   1.042]
 [44.25   1.093]] 44.25
[[21.75   1.042]
 [44.25   1.093]
 [38.25   1.052]] 38.25
[[44.25   1.093]
 [38.25   1.052]
 [21.75   1.052]] 21.75
[[38.25   1.052]
 [21.75   1.052]
 [20.75   1.052]] 20.75
[[21.75   1.052]
 [20.75   1.052]
 [34.25   1.052]] 34.25
[[20.75   1.052]
 [34.25   1.052]
 [48.     1.052]] 48.0
[[34.25   1.052]
 [48.     1.052]
 [48.5    1.075]] 48.5
[[48.     1.052]
 [48.5    1.075]
 [34.5    1.075]] 34.5
[[48.5    1.075]
 [34.5    1.075]
 [55.5    1.075]] 55.5
[[34.5    1.075]
 [55.5    1.075]
 [28.5    1.075]] 28.5
[[55.5    1.075]
 [28.5    1.075]
 [39.25   1.075]] 39.25
[[28.5    1.075]
 [39.25   1.075]
 [33.5    1.052]] 33.5
[[39.25   1.075]
 [33.5    1.052]
 [29.5    1.075]] 29.5
[[33.5    1.052]
 [29.5    1.075]
 [36.     1.075]] 36.0
[[29.5    1.0

2022-09-10 21:42:50.945458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-10 21:42:51.370256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Daniel Theis
[[ 5.5    1.042]
 [ 9.25   1.042]
 [13.     1.093]] 13.0
[[ 9.25   1.042]
 [13.     1.093]
 [ 4.75   1.052]] 4.75
[[13.     1.093]
 [ 4.75   1.052]
 [ 7.75   1.052]] 7.75
[[4.75  1.052]
 [7.75  1.052]
 [5.25  1.052]] 5.25
[[7.75  1.052]
 [5.25  1.052]
 [2.25  1.075]] 2.25
[[ 5.25   1.052]
 [ 2.25   1.075]
 [14.75   1.075]] 14.75
[[ 2.25   1.075]
 [14.75   1.075]
 [ 0.75   1.075]] 0.75
[[14.75   1.075]
 [ 0.75   1.075]
 [ 6.5    1.075]] 6.5
[[ 0.75   1.075]
 [ 6.5    1.075]
 [18.     1.052]] 18.0
[[ 6.5    1.075]
 [18.     1.052]
 [ 9.75   1.075]] 9.75
[[18.     1.052]
 [ 9.75   1.075]
 [ 6.     1.075]] 6.0
[[ 9.75   1.075]
 [ 6.     1.075]
 [20.5    1.093]] 20.5
[[ 6.     1.075]
 [20.5    1.093]
 [26.5    1.093]] 26.5
[[20.5    1.093]
 [26.5    1.093]
 [20.     1.093]] 20.0


...

2022-09-10 21:43:33.413603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-10 21:43:33.823156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Robert Williams III
[[30.75   1.042]
 [24.5    1.042]
 [34.     1.042]] 34.0
[[24.5    1.042]
 [34.     1.042]
 [10.     1.042]] 10.0
[[34.     1.042]
 [10.     1.042]
 [23.     1.042]] 23.0
[[10.     1.042]
 [23.     1.042]
 [26.5    1.052]] 26.5
[[23.     1.042]
 [26.5    1.052]
 [17.5    1.052]] 17.5
[[26.5    1.052]
 [17.5    1.052]
 [18.75   1.052]] 18.75
[[17.5    1.052]
 [18.75   1.052]
 [27.25   1.052]] 27.25
[[18.75   1.052]
 [27.25   1.052]
 [32.25   1.075]] 32.25
[[27.25   1.052]
 [32.25   1.075]
 [34.25   1.052]] 34.25
[[32.25   1.075]
 [34.25   1.052]
 [20.     1.075]] 20.0
[[34.25   1.052]
 [20.     1.075]
 [ 6.75   1.052]] 6.75
[[20.     1.075]
 [ 6.75   1.052]
 [24.     1.075]] 24.0
[[ 6.75   1.052]
 [24.     1.075]
 [34.5    1.042]] 34.5
[[24.     1.075]
 [34.5    1.042]
 [10.75   1.093]] 10.75
[[34.5    1.042]
 [10.75   1.093]
 [ 8.     1.093]] 8.0


2022-09-10 21:43:37.176371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-10 21:43:37.589218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Sam Hauser
[[0.    1.042]
 [0.    1.042]
 [5.    1.042]] 5.0
[[0.    1.042]
 [5.    1.042]
 [1.5   1.042]] 1.5
[[5.    1.042]
 [1.5   1.042]
 [3.25  1.075]] 3.25
[[1.5   1.042]
 [3.25  1.075]
 [0.    1.075]] 0.0
[[3.25  1.075]
 [0.    1.075]
 [0.    1.042]] 0.0


2022-09-10 21:43:41.018920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-10 21:43:41.829910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-10 21:43:45.211773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [180]:
add

{'Aaron Nesmith': 1.1010284423828125,
 'Al Horford': 15.41996955871582,
 'Daniel Theis': 20.017879486083984,
 'Derrick White': 12.299777030944824,
 'Grant Williams': 16.760189056396484,
 'Jaylen Brown': 38.3687858581543,
 'Jayson Tatum': 65.13274383544922,
 'Juwan Morgan': -0.02259661629796028,
 'Luke Kornet': -0.05909275263547897,
 'Malik Fitts': 0.002770889550447464,
 'Marcus Smart': 30.925310134887695,
 'Nik Stauskas': 0.03696215897798538,
 'Payton Pritchard': 14.317920684814453,
 'Robert Williams III': 8.039493560791016,
 'Sam Hauser': -0.002224911004304886}

# Stacked LSTM

In [42]:
add = {}

from keras.layers import Bidirectional

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 3
    if len(rent_rate) < 4:
        continue
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_steps, n_features))))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(rent_rate[-3:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Anthony Davis
Austin Reaves
[72.   36.75 17.75] 20.5
[36.75 17.75 20.5 ] 7.5
[17.75 20.5   7.5 ] 21.25
[20.5   7.5  21.25] 20.5


2022-10-15 16:27:17.741200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Cole Swider
[15.75 16.   30.75] 11.0
[16.   30.75 11.  ] 14.5
[30.75 11.   14.5 ] 12.5


2022-10-15 16:27:25.078307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:27:26.443326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


D.J. Augustin
Damian Jones
[9.   8.   1.25] 11.0
[ 8.    1.25 11.  ] 11.5
[ 1.25 11.   11.5 ] 5.25


...

2022-10-15 16:28:58.666689: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:28:59.703576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Shaquille Harrison
Stanley Johnson
Talen Horton-Tucker
Thomas Bryant
[ 7.75  5.75 31.75] 12.25
[ 5.75 31.75 12.25] 13.5
[31.75 12.25 13.5 ] 14.25


2022-10-15 16:29:08.866311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:29:11.092037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Wayne Ellington
Wenyen Gabriel
[35.    6.25  8.5 ] 6.75
[6.25 8.5  6.75] 27.5
[ 8.5   6.75 27.5 ] 8.0
[ 6.75 27.5   8.  ] 10.75


2022-10-15 16:29:20.951066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:29:22.006645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:29:31.258317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [43]:
add

{'Austin Reaves': 19.070512771606445,
 'Cole Swider': 7.0764265060424805,
 'Damian Jones': 17.173702239990234,
 'Juan Toscano-Anderson': 30.316635131835938,
 'Kendrick Nunn': 114.31356048583984,
 'LeBron James': 11.993985176086426,
 'Matt Ryan': 57.7443733215332,
 'Max Christie': 27.625999450683594,
 'Patrick Beverley': 5.453878402709961,
 'Russell Westbrook': 1.5929194688796997,
 'Scotty Pippen Jr.': 4.623406410217285,
 'Thomas Bryant': 9.415352821350098,
 'Wenyen Gabriel': 13.238883018493652}

In [46]:
add = {}

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 2
    if len(rent_rate) < 3:
        continue
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True,input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(rent_rate[-2:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Anthony Davis
[46.75 30.75] 38.75


2022-10-15 16:33:22.587388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Austin Reaves
[72.   36.75] 17.75
[36.75 17.75] 20.5
[17.75 20.5 ] 7.5
[20.5  7.5] 21.25
[ 7.5  21.25] 20.5


2022-10-15 16:33:28.400792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:33:29.390096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Cole Swider
[15.75 16.  ] 30.75
[16.   30.75] 11.0
[30.75 11.  ] 14.5
[11.  14.5] 12.5


...

2022-10-15 16:34:57.899747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:34:58.897896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Shaquille Harrison
Stanley Johnson
Talen Horton-Tucker
Thomas Bryant
[7.75 5.75] 31.75
[ 5.75 31.75] 12.25
[31.75 12.25] 13.5
[12.25 13.5 ] 14.25


2022-10-15 16:35:05.450171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:35:06.973125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Wayne Ellington
Wenyen Gabriel
[35.    6.25] 8.5
[6.25 8.5 ] 6.75
[8.5  6.75] 27.5
[ 6.75 27.5 ] 8.0
[27.5  8. ] 10.75


2022-10-15 16:35:14.433232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:35:15.509974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 16:35:22.227311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [47]:
add

{'Anthony Davis': 27.944467544555664,
 'Austin Reaves': 8.387675285339355,
 'Cole Swider': 25.410066604614258,
 'Damian Jones': 8.89858627319336,
 'Jay Huff': 36.7367057800293,
 'Juan Toscano-Anderson': 7.188565254211426,
 'Kendrick Nunn': 87.73473358154297,
 'LeBron James': 1.9515291452407837,
 'Lonnie Walker IV': 29.177108764648438,
 'Matt Ryan': 27.828468322753906,
 'Max Christie': 13.685097694396973,
 'Patrick Beverley': 2.987292766571045,
 'Russell Westbrook': 1.0200003385543823,
 'Scotty Pippen Jr.': 49.392120361328125,
 'Thomas Bryant': 13.161378860473633,
 'Wenyen Gabriel': 18.043560028076172}